In [1]:
# Dependencies
import os
import time
import copy
import pickle
import pathlib
import numpy as np
from dataclasses import dataclass
import torch
from ruamel.yaml import YAML, dump, RoundTripDumper
import raisimGymTorch.algo.ppo.module as ppo_module

In [2]:
# This notebook will generate a csv folder beside the .pt file
# All csv files for miniDNN will be stored in this folder
HOME_PATH = '/home/willw/raisim/rsmGymA1/data/'
DATA_PATH = HOME_PATH + "/a1_locomotion"
EXP_DATE = "220715-172712"
ITER_NUM = "1400"
BASE_CHECKPOINT_PATH = f"{DATA_PATH}/{EXP_DATE}/{ITER_NUM}/"
logFiles = os.listdir(BASE_CHECKPOINT_PATH)
print(logFiles)

outDir = f'{BASE_CHECKPOINT_PATH}/csv/'
print(outDir)
if not os.path.exists(outDir):
    os.makedirs(outDir)
    os.makedirs(f'{BASE_CHECKPOINT_PATH}/gzbLog/')

['test_150722_183320.mp4', 'csv', 'gzbLog', 'MClogs', 'mean.csv', 'full.pt', 'test_log_150722_183937.csv', 'action.npy', 'test_log_150722_184023.csv', 'test_log_150722_183659.csv', 'test_log_150722_183818.csv', 'eval_log.csv', 'var.csv', 'observation.npy', 'test_log_150722_183320.csv', 'train_policy.mp4']
/home/willw/raisim/rsmGymA1/data//a1_locomotion/220715-172712/1400//csv/


In [3]:
ob_dim=38
act_dim=12
cfg = YAML().load(open(f"{DATA_PATH}/{EXP_DATE}/stage_0.yaml", 'r'))
actor = ppo_module.MLP(cfg['architecture']['policy_net'], torch.nn.LeakyReLU, ob_dim, act_dim)
actor.load_state_dict(torch.load(f"{BASE_CHECKPOINT_PATH}/full.pt")['actor_architecture_state_dict'])

<All keys matched successfully>

In [4]:
# inspect .pt file
actor_dict = torch.load(f'{BASE_CHECKPOINT_PATH}/full.pt', map_location=torch.device("cpu"))
for i in actor_dict.keys():
    for ky, im in actor_dict[i].items():
        if(hasattr(im, 'shape')):
            print(i,": ",ky,":", im.shape)
#     np.savetxt(f"{outDir}/{i}.csv", actor_dict[i], fmt ='%.4f', delimiter =',')

actor_architecture_state_dict :  architecture.0.weight : torch.Size([128, 38])
actor_architecture_state_dict :  architecture.0.bias : torch.Size([128])
actor_architecture_state_dict :  architecture.2.weight : torch.Size([128, 128])
actor_architecture_state_dict :  architecture.2.bias : torch.Size([128])
actor_architecture_state_dict :  architecture.4.weight : torch.Size([12, 128])
actor_architecture_state_dict :  architecture.4.bias : torch.Size([12])
actor_distribution_state_dict :  std : torch.Size([12])
critic_architecture_state_dict :  architecture.0.weight : torch.Size([128, 38])
critic_architecture_state_dict :  architecture.0.bias : torch.Size([128])
critic_architecture_state_dict :  architecture.2.weight : torch.Size([128, 128])
critic_architecture_state_dict :  architecture.2.bias : torch.Size([128])
critic_architecture_state_dict :  architecture.4.weight : torch.Size([1, 128])
critic_architecture_state_dict :  architecture.4.bias : torch.Size([1])


In [5]:
actor_dict = torch.load(f'{BASE_CHECKPOINT_PATH}/full.pt', map_location=torch.device("cpu"))['actor_architecture_state_dict']
for ky, im in actor_dict.items():
    np.savetxt(f"{outDir}/{ky}.csv", im, fmt ='%.4f', delimiter =',')
    print(ky,":", im.shape)

architecture.0.weight : torch.Size([128, 38])
architecture.0.bias : torch.Size([128])
architecture.2.weight : torch.Size([128, 128])
architecture.2.bias : torch.Size([128])
architecture.4.weight : torch.Size([12, 128])
architecture.4.bias : torch.Size([12])


In [6]:
obMean = np.loadtxt(f'{BASE_CHECKPOINT_PATH}/mean.csv', delimiter =',')
obVar = np.loadtxt(f'{BASE_CHECKPOINT_PATH}/var.csv', delimiter =',')
obStd = np.sqrt(obVar+1e-8)
jointPgain = [50] * 12
jointDgain = [0.2] * 12
actionMean = [-0.1, 0.75, -1.6, 0.1, 0.75, -1.6, -0.1, 0.75, -1.6, 0.1, 0.75, -1.6]
actionStd = [0.3] * 12
actionMrg = [10] * 12

In [9]:
if os.path.exists(f'{outDir}/obStd.csv'):
    os.remove(f'{outDir}/obStd.csv')
if os.path.exists(f'{outDir}/obMean.csv'):
    os.remove(f'{outDir}/obMean.csv')
np.savetxt(f'{outDir}/obMean.csv', obMean, fmt ='%.8f', delimiter =',')
np.savetxt(f'{outDir}/obStd.csv', obStd , fmt ='%.8f', delimiter =',')
np.savetxt(f"{outDir}/jointPgain.csv", jointPgain, fmt ='%.1f', delimiter =',')
np.savetxt(f"{outDir}/jointDgain.csv", jointDgain, fmt ='%.2f', delimiter =',')
np.savetxt(f"{outDir}/acMean.csv", actionMean, fmt ='%.3f', delimiter =',')
np.savetxt(f"{outDir}/acStd.csv", actionStd, fmt ='%.3f', delimiter =',')
np.savetxt(f"{outDir}/acMargin.csv", actionMrg, fmt ='%.5f', delimiter =',')

In [ ]:
# Verify that loaded network works the same as training 
obSet = np.load(f'{BASE_CHECKPOINT_PATH}/observation.npy')[:,0,:]
acSet = np.load(f'{BASE_CHECKPOINT_PATH}/action.npy')[:,0,:]
for obs, ac in zip(obSet, acSet):
    action_ll = actor.architecture(torch.from_numpy(obs).cpu())
    print("pyt:", action_ll.detach().numpy())
    print("set:", ac)

In [59]:
np.savetxt(f"{outDir}/obSet.csv", obSet, fmt ='%.4f', delimiter =',')
np.savetxt(f"{outDir}/acSet.csv", acSet, fmt ='%.4f', delimiter =',')

In [10]:
# acSet=np.array([[-0.212521, 0.649908, -1.67029, -0.0393272, 0.758687, -1.50619, 0.0717264, 0.814442, -1.33921, 0.253062, 0.865441, -1.7184],[-0.117631, 0.679932, -1.6403, 0.0365807, 0.75857, -1.51355, -0.0371678, 0.805019, -1.42276, 0.167428, 0.856029, -1.63371],[-0.136347, 0.666971, -1.63042, 0.0405666, 0.757564, -1.51585, -0.0489819, 0.806666, -1.41241, 0.209161, 0.883913, -1.63646],[-0.194831, 0.634419, -1.63376, 0.0217506, 0.766067, -1.50526, -0.054111, 0.786701, -1.40138, 0.224769, 0.882672, -1.66742],[-0.175899, 0.615539, -1.65319, 0.0400725, 0.759866, -1.50276, -0.0610718, 0.767446, -1.39915, 0.199206, 0.853621, -1.65716],[-0.166413, 0.602716, -1.66346, 0.0435075, 0.754943, -1.49996, -0.077477, 0.746163, -1.39031, 0.184226, 0.842073, -1.65016],[-0.184863, 0.600891, -1.65576, 0.0524573, 0.749512, -1.49436, -0.0816343, 0.729684, -1.3768, 0.169678, 0.841975, -1.64763],[-0.200992, 0.604579, -1.6483, 0.0632243, 0.747495, -1.49087, -0.0930261, 0.70391, -1.36334, 0.155686, 0.824157, -1.64873],[-0.21329, 0.595125, -1.64803, 0.0681218, 0.745763, -1.48935, -0.102712, 0.670494, -1.34774, 0.155347, 0.796014, -1.64755],[-0.226755, 0.582396, -1.65381, 0.0660149, 0.743175, -1.48779, -0.112258, 0.634916, -1.33681, 0.152803, 0.766556, -1.64925],[-0.234408, 0.564164, -1.66683, 0.0624235, 0.743345, -1.49001, -0.118926, 0.607688, -1.33369, 0.138471, 0.734875, -1.65535],[-0.239981, 0.54824, -1.68281, 0.0597091, 0.736051, -1.49813, -0.120199, 0.577764, -1.33183, 0.113553, 0.704663, -1.66094],[-0.240316, 0.54227, -1.69995, 0.0570994, 0.724674, -1.50636, -0.124687, 0.538574, -1.33166, 0.0856016, 0.675309, -1.66977],[-0.228546, 0.554117, -1.7076, 0.0565796, 0.70784, -1.52777, -0.138183, 0.487326, -1.34221, 0.0305442, 0.651834, -1.67886],[-0.219142, 0.578273, -1.69727, 0.0548556, 0.684384, -1.55902, -0.160644, 0.41349, -1.3557, -0.0543466, 0.630155, -1.67547],[-0.237122, 0.584388, -1.68631, 0.0435751, 0.662574, -1.59586, -0.152592, 0.339119, -1.35689, -0.154133, 0.616449, -1.6585],[-0.26097, 0.630782, -1.59422, 0.0393522, 0.68283, -1.68495, -0.11552, 0.275694, -1.46937, -0.299967, 0.625038, -1.54417],[-0.238033, 0.588077, -1.56907, -0.0555911, 0.726989, -1.72125, -0.0542388, 0.288439, -1.57959, -0.376083, 0.670892, -1.38816],[-0.22572, 0.570936, -1.5413, -0.0913864, 0.777001, -1.69937, -0.0119299, 0.375601, -1.60944, -0.377229, 0.713081, -1.26284],[-0.185868, 0.576441, -1.38652, -0.0111414, 0.806443, -1.72166, -0.0163105, 0.451253, -1.71726, -0.397507, 0.65045, -1.22587],[-0.18201, 0.577364, -1.40375, 0.031064, 0.777942, -1.72704, -0.0401024, 0.495851, -1.7202, -0.367804, 0.59674, -1.2605],[-0.21592, 0.604957, -1.42085, 0.0864672, 0.749089, -1.76035, 0.00312207, 0.429504, -1.78406, -0.306962, 0.568228, -1.28139],[-0.218908, 0.645383, -1.42096, 0.0794029, 0.711989, -1.77811, 0.0482558, 0.339941, -1.88674, -0.248758, 0.554341, -1.2713],[-0.226342, 0.684075, -1.40955, 0.0865205, 0.704342, -1.76655, 0.0848251, 0.343168, -1.93757, -0.227001, 0.544648, -1.27495],[-0.225557, 0.698607, -1.40772, 0.105914, 0.69441, -1.75656, 0.129836, 0.373823, -1.95471, -0.205641, 0.521734, -1.27661],[-0.218949, 0.687946, -1.43144, 0.138487, 0.688118, -1.76201, 0.156961, 0.432766, -1.94769, -0.182629, 0.485921, -1.26601],[-0.209439, 0.681986, -1.43984, 0.150475, 0.66951, -1.77305, 0.159687, 0.442653, -1.97334, -0.155819, 0.461886, -1.25612],[-0.203966, 0.685713, -1.44132, 0.156706, 0.654546, -1.77657, 0.179502, 0.437666, -2.0061, -0.128461, 0.445477, -1.24888],[-0.194715, 0.691102, -1.4339, 0.166134, 0.644404, -1.77206, 0.198738, 0.440241, -2.02301, -0.108083, 0.429006, -1.23847],[-0.184224, 0.698331, -1.42264, 0.179607, 0.637285, -1.7634, 0.196541, 0.455668, -2.03085, -0.0902352, 0.420152, -1.22645],[-0.172545, 0.704412, -1.4123, 0.192158, 0.629342, -1.75836, 0.194397, 0.464749, -2.03955, -0.0750077, 0.410305, -1.21344],[-0.170747, 0.706955, -1.41599, 0.191492, 0.62126, -1.74165, 0.194037, 0.48643, -2.03743, -0.0472068, 0.404323, -1.20905],[-0.156507, 0.709503, -1.41664, 0.189475, 0.605192, -1.73315, 0.189782, 0.504934, -2.04089, -0.020584, 0.398556, -1.193],[-0.239743, 0.3984, -1.72245, -0.0923918, 0.57875, -1.52971, 0.407097, 0.622872, -1.47341, -0.136754, 0.381701, -1.57653],[0.0979878, 0.609326, -1.54537, 0.0459157, 0.521041, -1.71996, 0.106375, 0.425738, -1.62179, -0.130298, 0.402942, -1.29386],[-0.0750302, 0.612824, -1.66158, -0.0181622, 0.539188, -1.65937, 0.247356, 0.453844, -1.52934, -0.129173, 0.579473, -1.55707],[-0.0124657, 0.591525, -1.58355, -0.0349339, 0.529372, -1.69213, 0.291295, 0.302745, -1.62601, -0.183481, 0.494964, -1.4253],[-0.109681, 0.54944, -1.65634, -0.0444864, 0.582335, -1.61703, 0.391157, 0.395171, -1.45412, -0.132879, 0.517153, -1.44717],[-0.163509, 0.561146, -1.64409, -0.016209, 0.645346, -1.54884, 0.291922, 0.491891, -1.40911, -0.101624, 0.546369, -1.41592],[-0.090959, 0.574902, -1.62706, 0.0201876, 0.621443, -1.60168, 0.276122, 0.455004, -1.49694, -0.0966994, 0.535742, -1.38458],[-0.136307, 0.564803, -1.66846, 0.0151217, 0.643284, -1.55722, 0.350276, 0.47004, -1.48057, -0.121055, 0.549744, -1.38469],[-0.157326, 0.595079, -1.67162, 0.0335701, 0.681548, -1.55121, 0.386318, 0.481915, -1.5219, -0.122657, 0.576272, -1.3663],[-0.147264, 0.63702, -1.67345, 0.0621108, 0.705739, -1.56162, 0.39352, 0.485576, -1.6003, -0.118236, 0.601063, -1.32885],[-0.138318, 0.676604, -1.68815, 0.0906444, 0.727388, -1.55144, 0.40823, 0.520698, -1.66739, -0.112152, 0.627884, -1.298],[-0.105837, 0.701212, -1.67887, 0.0999841, 0.738094, -1.52492, 0.384128, 0.557762, -1.75806, -0.0805354, 0.658501, -1.27111],[-0.0943084, 0.718688, -1.68248, 0.103336, 0.728268, -1.49428, 0.387809, 0.572564, -1.81874, -0.0425423, 0.683068, -1.27152],[-0.079329, 0.718402, -1.68073, 0.134602, 0.721652, -1.47696, 0.369833, 0.579271, -1.83782, -0.01298, 0.682895, -1.26438],[-0.0648602, 0.727741, -1.65488, 0.174895, 0.713839, -1.47306, 0.349134, 0.571106, -1.84723, -0.000731947, 0.665562, -1.24658],[-0.0490251, 0.745647, -1.623, 0.203952, 0.704561, -1.46081, 0.340876, 0.555202, -1.85926, 0.00455088, 0.661329, -1.21443],[-0.0498804, 0.768399, -1.6027, 0.219589, 0.696834, -1.39563, 0.326548, 0.532623, -1.85668, 0.0115047, 0.66857, -1.17398],[-0.0552822, 0.760105, -1.59612, 0.221694, 0.67806, -1.35235, 0.326202, 0.513982, -1.83455, 0.0173377, 0.673404, -1.14236],[-0.0645682, 0.754974, -1.63902, 0.210426, 0.644636, -1.35142, 0.320477, 0.482305, -1.8104, 0.0170796, 0.677107, -1.12721],[-0.0297671, 0.766049, -1.64898, 0.210473, 0.62611, -1.3746, 0.287925, 0.467635, -1.81093, 0.0208598, 0.687043, -1.0775],[-0.0113924, 0.775995, -1.64203, 0.26634, 0.631479, -1.33317, 0.220627, 0.457414, -1.80747, -0.0108762, 0.716073, -0.989058],[-0.0180632, 0.793045, -1.59699, 0.305542, 0.632403, -1.29369, 0.185186, 0.456292, -1.82412, -0.0388671, 0.73298, -0.934597],[-0.0567994, 0.772154, -1.58238, 0.253237, 0.598295, -1.27277, 0.220402, 0.483526, -1.77745, -0.0196407, 0.730834, -0.920073],[-0.0639166, 0.806133, -1.58512, 0.242652, 0.580793, -1.28727, 0.223647, 0.456579, -1.77309, -0.0060027, 0.737639, -0.897737],[-0.0756551, 0.837422, -1.58867, 0.212443, 0.573696, -1.26089, 0.205093, 0.446356, -1.73511, 0.0358517, 0.756736, -0.872883],[-0.126902, 0.857645, -1.66224, 0.196236, 0.589424, -1.25772, 0.251299, 0.439894, -1.696, 0.0381311, 0.758535, -0.889112],[-0.165182, 0.872648, -1.75965, 0.188572, 0.497895, -1.3424, 0.322492, 0.403322, -1.64724, 0.0911464, 0.669278, -1.18284],[0.0344982, 0.809626, -1.49981, 0.159146, 0.528166, -1.36344, 0.141172, 0.305352, -1.819, 0.0256907, 0.532526, -0.94673],[-0.221736, 0.92998, -1.64586, 0.159386, 0.650028, -1.27471, 0.220544, 0.438463, -1.6501, 0.027361, 0.59461, -1.01251],[-0.164349, 0.898049, -1.54296, 0.125466, 0.627621, -1.29002, 0.252853, 0.381339, -1.72666, 0.0478521, 0.564776, -0.988646],[-0.136728, 0.852012, -1.54172, 0.100312, 0.54983, -1.34184, 0.267162, 0.398807, -1.73594, 0.0743545, 0.500959, -0.95579],[-0.189746, 0.895505, -1.49566, 0.151449, 0.527483, -1.27237, 0.280451, 0.428045, -1.72695, 0.0750042, 0.491192, -0.961195],[-0.211661, 0.89139, -1.51004, 0.142164, 0.494605, -1.24154, 0.299061, 0.430088, -1.71185, 0.0829221, 0.492502, -0.952246],[-0.293951, 0.703471, -1.81903, 0.06919, 0.387793, -1.08843, 0.424064, 0.558074, -1.4172, 0.203702, 0.689249, -1.36978],[-0.0885589, 0.795687, -1.65453, 0.18552, 0.421632, -1.25307, 0.353144, 0.472796, -1.55187, 0.0535593, 0.740184, -1.1537],[-0.19168, 0.919452, -1.72988, 0.108854, 0.337787, -1.12249, 0.386754, 0.631997, -1.38659, 0.303282, 0.877353, -1.33575],[-0.258869, 0.954677, -1.78994, 0.14402, 0.42471, -1.09633, 0.401383, 0.588764, -1.36905, 0.266898, 0.932781, -1.27427],[-0.212004, 1.00673, -1.74561, 0.197436, 0.422004, -1.15416, 0.432184, 0.541236, -1.39171, 0.269182, 0.978811, -1.2629],[-0.234413, 1.09522, -1.74453, 0.171566, 0.271, -1.0749, 0.453946, 0.580041, -1.36881, 0.364717, 1.01793, -1.34516],[-0.289936, 1.11142, -1.81753, 0.176249, 0.274032, -1.09534, 0.532694, 0.571198, -1.33967, 0.383049, 1.02162, -1.33355],[-0.303531, 1.21686, -1.79616, 0.158689, 0.24421, -0.998406, 0.571242, 0.610878, -1.30295, 0.448199, 1.09251, -1.39026]])
# obSet=np.array([[0.285041, 0.0569189, 0.0145319, 0.998273, -0.121425, 0.697208, -1.64411, 0.0794766, 0.703559, -1.64448, -0.0613709, 0.712521, -1.74019, 0.114611, 0.697104, -1.7722, 0.0318736, -0.0496054, 0.00119616, 0.243064, 0.0278392, 0.233135, 0, 0, 0, -2.12045, -0.280339, -1.49877, -0.243618, 0.237057, -0.230449, 0.0447383, -0.12403, 0.257376, 3.18892, 3.75607, -2.3097, 0.06],[0.285293, 0.0560734, 0.0172441, 0.998278, -0.147798, 0.686891, -1.65045, 0.0755572, 0.706189, -1.64529, -0.0604955, 0.709416, -1.73528, 0.145346, 0.731137, -1.79052, 0.0365261, -0.0688921, 0.0379916, 0.259015, 0.0305261, 0.502739, 0, 0, 0, -2.44369, -1.50011, 0.369704, -0.421856, 0.258043, 0.0195674, 0.171898, -0.383892, 0.62406, 1.98287, 2.07992, -0.982828, 0.09],[0.285885, 0.0551642, 0.0170903, 0.998331, -0.164178, 0.671333, -1.63951, 0.073148, 0.708549, -1.64316, -0.0553781, 0.704946, -1.72808, 0.154743, 0.737392, -1.79113, 0.0466773, -0.0954477, 0.0656464, -0.317955, 0.172513, 0.696566, 0, 0, 0, -0.371891, -0.236607, -0.0970506, -0.00497936, 0.204572, 0.333255, 0.868751, -0.511954, 0.7633, 0.845092, 0.230673, 0.290482, 0.12],[0.286636, 0.0541537, 0.017033, 0.998387, -0.168773, 0.668557, -1.63692, 0.0697885, 0.711018, -1.63896, -0.0478115, 0.69793, -1.71749, 0.161947, 0.740343, -1.78804, 0.0543053, -0.10483, 0.0968244, 0.37167, 0.0394451, 1.0252, 0, 0, 0, -0.929601, -0.403247, 0.476483, -0.772675, 0.31456, 0.419571, 0.476376, -0.853223, 1.30719, 0.436983, 0.374489, 0.263676, 0.15],[0.2878, 0.0524989, 0.0232138, 0.998351, -0.182794, 0.659675, -1.631, 0.0581191, 0.714305, -1.63479, -0.0452141, 0.686459, -1.70055, 0.164035, 0.743608, -1.78445, 0.0552623, -0.115914, 0.131556, 0.740831, 0.109497, 1.28514, 0, 0, 0, -1.57982, -1.37584, 0.546369, -1.30254, 0.299833, 0.418965, 0.226067, -1.35868, 1.93929, 0.171086, 0.234273, 0.47506, 0.18],[0.289287, 0.0490076, 0.0302796, 0.998339, -0.198067, 0.640074, -1.63144, 0.044647, 0.716503, -1.63064, -0.0423471, 0.669623, -1.6777, 0.166241, 0.743702, -1.77704, 0.0536116, -0.128511, 0.161047, 0.726237, 0.333645, 1.43951, 0, 0, 0, -1.38969, -2.2821, -0.556548, -1.33377, 0.159898, 0.405655, 0.358028, -1.89412, 2.49959, 0.284898, -0.154328, 0.924365, 0.21],[0.291042, 0.043807, 0.0369702, 0.998356, -0.210661, 0.613767, -1.64092, 0.0311695, 0.71749, -1.62641, -0.0383246, 0.648306, -1.64992, 0.169457, 0.740388, -1.76563, 0.0559852, -0.139736, 0.186079, 0.764328, 0.45464, 1.5432, 0, 0, 0, -1.22186, -2.80911, -1.03955, -1.38022, 0.0733741, 0.440036, 0.400161, -2.25284, 2.93245, 0.322067, -0.413147, 1.24741, 0.24],[0.293018, 0.038005, 0.0447695, 0.998274, -0.223056, 0.585525, -1.64982, 0.0164964, 0.718102, -1.62165, -0.0350722, 0.624604, -1.61862, 0.171997, 0.735603, -1.75213, 0.0624186, -0.148895, 0.205372, 0.929022, 0.467085, 1.59443, 0, 0, 0, -1.28077, -2.73788, -0.715941, -1.53951, 0.0616831, 0.497558, 0.255363, -2.42371, 3.23834, 0.192023, -0.500192, 1.38901, 0.27],[0.295154, 0.0320859, 0.0538313, 0.998034, -0.235459, 0.56129, -1.65456, 0.000821139, 0.718628, -1.6163, -0.0330084, 0.599731, -1.58483, 0.17354, 0.730124, -1.73746, 0.0682813, -0.15881, 0.218874, 0.978368, 0.490663, 1.59547, 0, 0, 0, -1.18771, -2.1706, -0.334667, -1.56197, 0.0419021, 0.553307, 0.192684, -2.52474, 3.45227, 0.145472, -0.588205, 1.52112, 0.3],[0.297384, 0.0259278, 0.063358, 0.997654, -0.247002, 0.544565, -1.65818, -0.014791, 0.718916, -1.61078, -0.0315026, 0.57391, -1.54919, 0.174672, 0.72358, -1.72153, 0.0721051, -0.167307, 0.224364, 1.0171, 0.531004, 1.56577, 0, 0, 0, -1.109, -1.31976, -0.417531, -1.54452, 0.0198363, 0.538187, 0.133364, -2.61539, 3.62029, 0.10067, -0.702591, 1.63669, 0.33],[0.299601, 0.0193223, 0.0736236, 0.997099, -0.257564, 0.536286, -1.66547, -0.0303991, 0.719061, -1.60626, -0.0310303, 0.547194, -1.51217, 0.17489, 0.715725, -1.70475, 0.073539, -0.17274, 0.216543, 1.09683, 0.580294, 1.4928, 0, 0, 0, -0.971671, -0.526036, -0.907625, -1.54419, 0.0186298, 0.39551, 0.0119022, -2.67651, 3.71339, -0.00573972, -0.81709, 1.67787, 0.36],[0.301658, 0.0123072, 0.0849441, 0.99631, -0.266042, 0.534188, -1.67796, -0.0460915, 0.719387, -1.60393, -0.0321313, 0.520273, -1.47492, 0.173663, 0.70692, -1.6882, 0.0719754, -0.17465, 0.194107, 1.21637, 0.628564, 1.36767, 0, 0, 0, -0.715185, -0.0580922, -1.38506, -1.56084, 0.0467395, 0.128662, -0.171488, -2.6722, 3.6906, -0.173271, -0.899789, 1.60963, 0.39],[0.303409, 0.00491561, 0.0977982, 0.995194, -0.271084, 0.535168, -1.69427, -0.0621506, 0.720166, -1.60498, -0.0355277, 0.493858, -1.43867, 0.168948, 0.697463, -1.67306, 0.0659458, -0.172466, 0.156412, 1.39397, 0.662819, 1.18354, 0, 0, 0, -0.318752, 0.159879, -1.6851, -1.6143, 0.106772, -0.257682, -0.435122, -2.5815, 3.52863, -0.764484, -0.956959, 1.42588, 0.42],[0.304653, -0.00248774, 0.114103, 0.993466, -0.271596, 0.538076, -1.70948, -0.080187, 0.722136, -1.61117, -0.043511, 0.469419, -1.40533, 0.14792, 0.687457, -1.66088, 0.0552733, -0.163499, 0.0952356, 1.83182, 0.6365, 0.8943, 0, 0, 0, 0.208414, 0.375318, -1.30272, -1.88747, 0.258786, -0.843499, -0.996794, -2.31048, 3.14695, -3.09911, -1.07633, 1.07443, 0.45],[0.305211, -0.00983318, 0.136746, 0.990557, -0.266919, 0.543203, -1.7136, -0.102865, 0.725841, -1.62464, -0.0593231, 0.447739, -1.37595, 0.0946571, 0.674221, -1.65149, 0.035574, -0.148816, 0.0213043, 2.56269, 0.636163, 0.527149, 0, 0, 0, 0.73667, 0.637489, 0.214039, -2.43529, 0.430058, -1.65109, -1.8739, -2.04978, 2.75339, -6.77761, -1.59566, 0.927355, 0.48],[0.305015, -0.0178202, 0.166517, 0.985878, -0.260183, 0.55069, -1.7034, -0.131099, 0.730288, -1.64421, -0.0810269, 0.429591, -1.3522, 0.00253863, 0.656254, -1.64047, 0.0046425, -0.136272, -0.0542321, 3.19399, 0.731766, 0.182249, 0, 0, 0, 0.780846, 0.86144, 1.60222, -2.91829, 0.5108, -2.47214, -2.65604, -1.99094, 2.532, -10.6206, -2.3106, 1.51927, 0.51],[0.304156, -0.0266445, 0.196833, 0.980075, -0.246955, 0.5588, -1.66266, -0.144391, 0.739282, -1.67759, -0.114483, 0.41146, -1.36069, -0.109597, 0.629482, -1.61785, -0.0158713, -0.126901, -0.111202, 2.85216, 0.812768, -0.141924, 0, 0, 0, 1.49402, 0.876768, 6.09053, 0.0917864, 1.0694, -3.99591, -3.19641, -1.80106, -3.96637, -11.2034, -2.56813, 2.68034, 0.54],[0.302793, -0.0369357, 0.21796, 0.975259, -0.225357, 0.566042, -1.5781, -0.125171, 0.748566, -1.7225, -0.138898, 0.395436, -1.47197, -0.202758, 0.600716, -1.58249, -0.021595, -0.109887, -0.157269, 1.70024, 1.10095, -0.269496, 0, 0, 0, 2.48136, 0.710342, 7.92603, 2.67643, 1.03809, -4.68977, -1.71852, -1.58614, -13.5089, -7.71465, -2.95238, 3.18306, 0.57],[0.301178, -0.0503867, 0.225312, 0.972983, -0.2008, 0.575915, -1.53425, -0.0992163, 0.754193, -1.7678, -0.136796, 0.37518, -1.61122, -0.238689, 0.575089, -1.56104, -0.00865189, -0.0852446, -0.158142, 0.36253, 1.43912, -0.247048, 0, 0, 0, 0.736855, 1.06507, -3.6738, 1.87818, 0.191252, -3.51631, 1.43441, -2.73067, -12.7606, -1.59022, -2.05891, 1.06335, 0.6],[0.299676, -0.0642707, 0.221449, 0.973052, -0.190542, 0.576445, -1.55789, -0.076826, 0.752977, -1.79659, -0.0980451, 0.353442, -1.7395, -0.235949, 0.556702, -1.55275, 0.00416514, -0.0724969, -0.116972, -0.586354, 1.11613, -0.585418, 0, 0, 0, 1.32245, 0.183682, -2.18026, 2.52863, -0.212976, -2.46922, 4.76135, -0.938284, -12.509, 0.625644, -1.75819, 0.86833, 0.63],[0.2988, -0.0739782, 0.215863, 0.973617, -0.176264, 0.578575, -1.57479, -0.0566746, 0.748866, -1.81599, -0.043982, 0.353954, -1.82746, -0.231206, 0.539165, -1.543, 0.00289082, -0.0588291, -0.0607094, -0.25538, 0.895257, -0.996804, 0, 0, 0, 1.37782, 0.195511, -1.31543, 1.59714, -0.538248, -1.65031, 5.43278, 0.771806, -5.82681, 0.226002, -1.78815, 1.07133, 0.66],[0.298475, -0.0821165, 0.210991, 0.974033, -0.160925, 0.580488, -1.58386, -0.0374994, 0.742367, -1.82982, 0.0104077, 0.367994, -1.84698, -0.229535, 0.520158, -1.53011, -0.00542639, -0.0536317, -0.0117966, -0.477898, 0.804179, -1.31332, 0, 0, 0, 1.73911, 0.191147, -0.696292, 2.26889, -0.734446, -1.21746, 5.43552, 1.65588, 0.00338077, 0.211676, -2.01738, 1.48713, 0.69],[0.298513, -0.0890707, 0.204058, 0.974898, -0.140704, 0.582455, -1.58896, -0.0137287, 0.733086, -1.83946, 0.0597237, 0.385348, -1.84608, -0.226437, 0.497849, -1.5123, -0.0157235, -0.0434588, 0.0157186, -0.831052, 0.841761, -1.53118, 0, 0, 0, 2.22699, 0.201937, -0.428361, 2.44232, -1.06389, -0.803581, 4.53739, 1.64122, -0.861788, 0.396322, -2.38303, 1.98533, 0.72],[0.298805, -0.0960587, 0.194947, 0.976099, -0.11767, 0.584825, -1.59082, 0.0113218, 0.721813, -1.84533, 0.096266, 0.401709, -1.87809, -0.223037, 0.473603, -1.49131, -0.0184172, -0.0266107, 0.0384859, -0.788764, 0.690453, -1.73963, 0, 0, 0, 2.29191, 0.267992, 0.0122289, 2.48412, -1.12995, -0.443432, 3.06674, 1.61439, -4.8171, 0.247829, -2.39142, 2.11485, 0.75],[0.299294, -0.100719, 0.186597, 0.97726, -0.0949434, 0.588087, -1.58745, 0.0359593, 0.710891, -1.84712, 0.122441, 0.414162, -1.93597, -0.221379, 0.451063, -1.47073, -0.0201142, -0.0150996, 0.0561729, -0.792554, 0.425744, -1.86872, 0, 0, 0, 2.28927, 0.374094, 0.547184, 2.47218, -1.05726, 0.0195817, 2.54371, 0.928921, -5.91298, 0.151431, -2.13901, 2.00314, 0.78],[0.299933, -0.10265, 0.177407, 0.97877, -0.0712739, 0.592532, -1.57937, 0.0615413, 0.700829, -1.84419, 0.149641, 0.419253, -1.98102, -0.219291, 0.431093, -1.45121, -0.0218732, -0.00914562, 0.0685364, -0.984316, 0.220234, -1.95585, 0, 0, 0, 2.45387, 0.492858, 0.955625, 2.63756, -0.984658, 0.47714, 2.92445, 0.384811, -3.24707, 0.278675, -1.91682, 1.92355, 0.81],[0.300669, -0.102679, 0.168384, 0.980359, -0.0478106, 0.59773, -1.56755, 0.0868717, 0.69149, -1.83796, 0.175628, 0.422959, -1.99769, -0.217745, 0.412816, -1.43254, -0.0216374, 0.00151475, 0.0763328, -0.768441, 0.0653651, -2.07103, 0, 0, 0, 2.22339, 0.527266, 1.31944, 2.41315, -0.904368, 0.686476, 2.19334, 0.459117, -0.915578, 0.0163101, -1.78408, 1.82775, 0.84],[0.301426, -0.101591, 0.162114, 0.981529, -0.0275057, 0.603196, -1.55282, 0.109211, 0.683085, -1.83059, 0.190553, 0.429923, -2.00398, -0.219494, 0.395711, -1.41506, -0.0191, 0.0151972, 0.0728191, -0.565016, -0.056358, -2.15352, 0, 0, 0, 1.93845, 0.568915, 1.54007, 2.1507, -0.799682, 0.767332, 1.08401, 0.876341, -0.754547, -0.264964, -1.66393, 1.69662, 0.87],[0.302098, -0.0995345, 0.156539, 0.982644, -0.00853227, 0.609358, -1.53694, 0.130439, 0.675712, -1.82235, 0.197656, 0.440625, -2.01474, -0.22229, 0.379798, -1.39878, -0.0175526, 0.0279039, 0.0611713, -0.650545, -0.125443, -2.16834, 0, 0, 0, 1.91663, 0.657617, 1.58575, 2.14925, -0.701291, 0.871748, 0.590618, 1.1597, -1.30553, -0.236976, -1.54439, 1.58629, 0.9],[0.302643, -0.0965018, 0.149842, 0.983989, 0.0107994, 0.616464, -1.52097, 0.152183, 0.66931, -1.81294, 0.201203, 0.453152, -2.02925, -0.22406, 0.365144, -1.3836, -0.0158444, 0.0395188, 0.0485735, -0.752434, -0.190145, -2.1564, 0, 0, 0, 1.91542, 0.744615, 1.59976, 2.16066, -0.597134, 0.979489, 0.169826, 1.30339, -1.47444, -0.160881, -1.40554, 1.45856, 0.93],[0.303067, -0.0926034, 0.142372, 0.985472, 0.030002, 0.62429, -1.50506, 0.173884, 0.663869, -1.8026, 0.200286, 0.465995, -2.04265, -0.225072, 0.351756, -1.36958, -0.0151023, 0.0512656, 0.0362769, -0.832073, -0.232477, -2.11823, 0, 0, 0, 1.88014, 0.803896, 1.57863, 2.13168, -0.506076, 1.05383, -0.28585, 1.20517, -1.13926, -0.0986634, -1.28576, 1.34255, 0.96],[0.303355, -0.0878059, 0.134589, 0.987004, 0.0469378, 0.632942, -1.49086, 0.194634, 0.659515, -1.79151, 0.195853, 0.477712, -2.0484, -0.225646, 0.340121, -1.35737, -0.0128308, 0.0638659, 0.022285, -0.874575, -0.323704, -2.0331, 0, 0, 0, 1.39557, 0.930605, 1.19538, 2.00198, -0.372956, 1.1338, -0.548787, 1.15663, -0.149918, -0.0737713, -1.06211, 1.12025, 0.99],[0.303514, -0.081841, 0.125749, 0.988681, 0.056972, 0.643598, -1.48391, 0.214899, 0.656351, -1.77922, 0.190123, 0.490838, -2.04635, -0.22503, 0.330666, -1.34704, -0.0114748, 0.0768743, 0.0102685, -1.12396, -0.347746, -1.92029, 0, 0, 0, 0.760862, 1.14785, 0.365295, 2.04795, -0.278798, 1.28289, -0.579301, 1.42461, 0.374896, 0.150232, -0.868888, 0.973063, 0.0199995],[0.303569, -0.0702207, 0.138238, 0.987907, -0.00573106, 0.651282, -1.5592, 0.205987, 0.658864, -1.76599, 0.214834, 0.470724, -1.85162, -0.228429, 0.378357, -1.47137, -0.034059, 0.115457, -0.00373923, 2.09867, -2.20531, -1.07383, 0, 0, 0, -10.4151, 0.00629998, -11.3104, -2.53421, 0.751465, 1.05342, 3.6598, -2.09027, 23.824, -0.483004, 7.48937, -18.7612, 0.0499995],[0.30375, -0.0595642, 0.158363, 0.985583, -0.0870886, 0.623101, -1.61992, 0.186778, 0.662524, -1.75886, 0.235104, 0.44488, -1.63354, -0.225309, 0.403829, -1.53765, -0.0914724, 0.141409, 0.0362019, 1.2529, -1.26804, -1.10426, 0, 0, 0, -6.54179, -5.28288, -2.41019, -1.49021, -0.0542196, 0.914087, 1.51282, -3.6404, 23.9971, 1.08203, -1.14418, 0.82582, 0.0799995],[0.303661, -0.0489427, 0.171207, 0.984019, -0.133888, 0.566911, -1.63607, 0.175437, 0.663542, -1.749, 0.24238, 0.422106, -1.45013, -0.202593, 0.415237, -1.51528, -0.057564, 0.151382, -0.0588257, 0.619967, -1.65222, -0.922934, 0, 0, 0, -4.9313, -4.70439, -2.67405, -1.22513, 0.428706, 1.03348, 0.581043, -0.998156, 16.5474, 3.42873, 4.44619, -2.57352, 0.11],[0.302542, -0.033956, 0.175567, 0.983882, -0.159946, 0.522875, -1.6421, 0.169516, 0.669708, -1.73814, 0.248949, 0.447034, -1.40931, -0.162363, 0.466382, -1.52067, -0.0720292, 0.192806, -0.162513, -0.412354, -1.53731, -1.00876, 0, 0, 0, -0.604446, -4.56558, 1.7779, -0.0972271, 0.608631, 1.0564, 0.811924, 4.26309, -4.37281, 3.93397, 3.92754, 3.81502, 0.14],[0.300533, -0.0171749, 0.170333, 0.985237, -0.151852, 0.489405, -1.62904, 0.172486, 0.67686, -1.72765, 0.273325, 0.480446, -1.46943, -0.165472, 0.48692, -1.54945, -0.100471, 0.21611, -0.217689, -1.56977, -1.22588, -0.729648, 0, 0, 0, 1.45481, -2.29698, -0.404196, 0.738081, 0.660646, 1.10812, 4.19441, 1.43194, -3.22356, 0.0992862, 1.80293, -2.37554, 0.17],[0.298432, 0.000482697, 0.164656, 0.986351, -0.146587, 0.482251, -1.65247, 0.175122, 0.684501, -1.71525, 0.321111, 0.472842, -1.45262, -0.168001, 0.508276, -1.57721, -0.101046, 0.251028, -0.199452, -0.591853, -1.75215, -0.575678, 0, 0, 0, -0.747817, 0.345712, -2.63115, -0.44471, 0.773681, 1.11522, 3.45541, -1.39859, 3.97633, -0.847935, 2.09773, -2.70095, 0.2],[0.296357, 0.0204382, 0.164071, 0.986237, -0.151575, 0.49344, -1.66886, 0.17021, 0.691329, -1.70242, 0.344332, 0.467066, -1.43553, -0.177684, 0.527474, -1.60201, -0.110381, 0.288865, -0.181328, -0.925716, -1.48187, -0.704131, 0, 0, 0, 0.458778, 1.45764, -0.576576, -0.224184, 0.521137, 1.23886, 1.29408, 0.327312, -1.2455, -0.763596, 1.65575, -2.18424, 0.23],[0.294825, 0.0351275, 0.157724, 0.986858, -0.143276, 0.514848, -1.67215, 0.171622, 0.694005, -1.69004, 0.353469, 0.475413, -1.46908, -0.18209, 0.540054, -1.61911, -0.124206, 0.3237, -0.160324, -1.4086, -1.14811, -0.750669, 0, 0, 0, 1.00728, 2.62611, -0.659015, 0.34213, 0.115536, 1.44217, 0.98184, 0.798813, -3.68325, -0.244521, 1.02165, -1.41632, 0.26],[0.293354, 0.0502209, 0.145147, 0.988135, -0.134229, 0.548104, -1.68239, 0.177978, 0.69309, -1.67404, 0.364712, 0.482927, -1.50964, -0.181585, 0.547245, -1.62921, -0.134049, 0.364583, -0.137525, -1.92761, -0.844411, -0.782591, 0, 0, 0, 0.734423, 3.75849, -1.15992, 0.806407, -0.220065, 1.72722, 1.29172, 0.74874, -4.31759, 0.239611, 0.551629, -0.773876, 0.29],[0.292082, 0.0642982, 0.12624, 0.989914, -0.128026, 0.590896, -1.6922, 0.189539, 0.688957, -1.65524, 0.379298, 0.493253, -1.56667, -0.175821, 0.549764, -1.63278, -0.136217, 0.406961, -0.119816, -2.53261, -0.46047, -0.867759, 0, 0, 0, 0.593529, 4.58691, -0.762867, 1.39, -0.553461, 1.98633, 1.53016, 1.26075, -6.85447, 0.811994, 0.0462609, -0.0841586, 0.32],[0.290963, 0.07691, 0.100077, 0.992003, -0.122209, 0.640968, -1.69805, 0.207376, 0.681501, -1.63403, 0.395895, 0.509334, -1.64859, -0.163946, 0.547066, -1.62924, -0.130652, 0.447637, -0.106154, -3.16864, -0.0334375, -0.98265, 0, 0, 0, 0.575676, 5.28518, -0.553486, 2.03544, -0.870222, 2.21459, 1.7539, 1.83633, -8.90024, 1.43739, -0.466481, 0.62711, 0.35],[0.290006, 0.0881984, 0.0671237, 0.993839, -0.113666, 0.692602, -1.69364, 0.230973, 0.671064, -1.61024, 0.409227, 0.53508, -1.75407, -0.146406, 0.53964, -1.61881, -0.120137, 0.485962, -0.0856053, -3.57655, 0.30149, -1.08409, 0, 0, 0, 0.96924, 4.88243, 1.26144, 2.45025, -1.14698, 2.43742, 0.866012, 2.78386, -10.9088, 1.84654, -0.922315, 1.29858, 0.38],[0.289269, 0.098097, 0.0308883, 0.994697, -0.102875, 0.736141, -1.67427, 0.256277, 0.658132, -1.58471, 0.412082, 0.559783, -1.85246, -0.126314, 0.528397, -1.60257, -0.114667, 0.519862, -0.0645737, -3.73906, 0.536006, -1.01292, 0, 0, 0, 1.03676, 3.96314, 2.16952, 2.50008, -1.36688, 2.58175, -0.0928559, 1.98632, -8.41207, 2.0385, -1.2398, 1.81725, 0.41],[0.288737, 0.10626, -0.00670594, 0.994316, -0.092275, 0.767253, -1.65295, 0.280886, 0.643072, -1.55874, 0.405795, 0.567682, -1.90067, -0.105099, 0.513891, -1.58125, -0.115672, 0.550459, -0.046268, -3.74622, 0.781204, -0.847224, 0, 0, 0, 1.02275, 2.51595, 1.94505, 2.38221, -1.5839, 2.57471, -0.998798, -0.0236661, -2.27217, 2.12459, -1.57799, 2.31737, 0.44],[0.288334, 0.112234, -0.0446987, 0.992676, -0.0820307, 0.780655, -1.63389, 0.301772, 0.627849, -1.53612, 0.391558, 0.558847, -1.89333, -0.0858132, 0.497797, -1.55781, -0.112191, 0.58301, -0.039529, -3.69184, 1.00999, -0.718479, 0, 0, 0, 1.21484, 0.860785, 2.21904, 2.26959, -1.75044, 2.46998, -1.96642, -1.43707, 2.40206, 2.14091, -1.90516, 2.75655, 0.47],[0.287887, 0.116636, -0.084106, 0.989607, -0.0673645, 0.78046, -1.60941, 0.324375, 0.609688, -1.51182, 0.366902, 0.542591, -1.85851, -0.063795, 0.477382, -1.52801, -0.0926137, 0.618137, -0.05434, -3.8813, 1.21247, -0.686574, 0, 0, 0, 1.67806, -0.482945, 2.59111, 2.24317, -1.87269, 2.42363, -2.72902, -1.52827, 3.53487, 2.27493, -2.101, 3.0891, 0.5],[0.287279, 0.121243, -0.12576, 0.984624, -0.045018, 0.781674, -1.61606, 0.347031, 0.591201, -1.48634, 0.336475, 0.531609, -1.83166, -0.040403, 0.456881, -1.49641, -0.0626406, 0.658528, -0.0658396, -4.06415, 1.14062, -0.623028, 0, 0, 0, 2.44995, 0.204393, -1.38197, 2.27267, -1.77201, 2.62891, -3.26061, -0.784544, 1.96198, 2.32725, -1.94586, 3.13938, 0.529999],[0.286613, 0.128241, -0.165644, 0.977812, -0.0218168, 0.784026, -1.62714, 0.367844, 0.574327, -1.45442, 0.302096, 0.525158, -1.81573, -0.0194767, 0.440245, -1.46652, -0.0358541, 0.707069, -0.0691442, -3.89589, 0.749614, -0.400095, 0, 0, 0, 1.82077, 0.218993, -0.90583, 1.58046, -1.61562, 3.08014, -3.43429, -0.572029, 1.34873, 1.91091, -1.44417, 2.84951, 0.559999],[0.285927, 0.136771, -0.196323, 0.970954, -0.0158022, 0.783647, -1.63334, 0.376344, 0.559585, -1.42585, 0.268664, 0.515015, -1.79732, -0.00547228, 0.42954, -1.44118, -0.0122541, 0.755592, -0.0848816, -3.33012, 0.28706, -0.042978, 0, 0, 0, -0.209403, -0.279896, -0.417753, 0.534619, -1.36682, 3.18938, -3.21479, -1.52367, 2.50821, 1.02895, -0.820437, 2.30063, 0.589999],[0.284694, 0.148252, -0.227507, 0.962425, -0.0283946, 0.777782, -1.63823, 0.365354, 0.545012, -1.38872, 0.236574, 0.494437, -1.77418, 0.00155267, 0.423113, -1.41846, 0.0258013, 0.806458, -0.140932, -3.05321, 0.170945, 0.225059, 0, 0, 0, -1.43798, -0.640724, -0.571849, -2.34262, -0.966429, 3.4322, -2.63617, -1.9704, 1.41505, 0.508587, -0.490318, 1.95291, 0.619999],[0.283058, 0.157883, -0.257912, 0.953181, -0.0388266, 0.770293, -1.64211, 0.335182, 0.545319, -1.356, 0.21353, 0.476349, -1.77433, 0.00853212, 0.417591, -1.39721, 0.0608937, 0.847673, -0.176792, -3.62377, 0.483262, 0.133876, 0, 0, 0, -0.540684, -0.792004, -0.169655, -3.35526, 0.233715, 3.40052, -2.14057, -1.67846, -0.923655, 0.958185, -0.635152, 2.30018, 0.649999],[0.281207, 0.167042, -0.295926, 0.940492, -0.0334499, 0.769799, -1.647, 0.311574, 0.554516, -1.31812, 0.193101, 0.463534, -1.79785, 0.022233, 0.409622, -1.37031, 0.0959344, 0.892445, -0.186958, -4.33941, 0.72857, -0.0947015, 0, 0, 0, 1.06556, 0.45827, -0.869497, -1.58211, 0.990074, 4.50262, -2.12691, -0.998129, -3.16335, 1.58037, -0.869934, 2.91428, 0.679999],[0.279156, 0.177777, -0.340471, 0.923295, -0.0232002, 0.774265, -1.65391, 0.29216, 0.570482, -1.27489, 0.174426, 0.452795, -1.81511, 0.0378131, 0.400409, -1.33881, 0.12423, 0.951675, -0.188151, -4.35739, 0.62192, -0.0698026, 0, 0, 0, 0.904369, 0.413757, -0.538477, -2.29975, 1.43833, 5.10888, -1.49404, -1.18745, -0.0642809, 1.46074, -0.915622, 3.259, 0.709999],[0.277356, 0.186913, -0.37494, 0.908011, -0.0167819, 0.778157, -1.65784, 0.269319, 0.597383, -1.23888, 0.171037, 0.452019, -1.8343, 0.0505625, 0.39177, -1.30572, 0.145898, 0.991452, -0.206171, -4.12219, 0.517184, 0.0712882, 0, 0, 0, 0.402375, 0.335508, -0.289148, -2.57916, 3.15877, 1.83642, 0.150533, 0.564733, -3.2099, 1.09469, -0.858527, 3.35861, 0.739999],[0.275294, 0.195992, -0.407768, 0.891803, -0.0184277, 0.779597, -1.65838, 0.23649, 0.626373, -1.23069, 0.166596, 0.455496, -1.86288, 0.0571215, 0.382502, -1.27069, 0.157272, 1.03557, -0.205775, -3.47542, 0.386644, 0.341658, 0, 0, 0, -0.51988, 0.00694111, 0.12568, -3.74987, 2.42673, 0.833353, -0.809135, 0.201276, -2.64206, 0.383784, -0.963529, 3.60424, 0.769999],[0.274671, 0.194322, -0.421036, 0.885984, -0.0553822, 0.779953, -1.66488, 0.177101, 0.64154, -1.23107, 0.143306, 0.443944, -1.86264, 0.0445146, 0.360994, -1.22086, 0.212185, 1.011, 0.0620021, 0.268243, 0.985655, 1.06474, 0, 0, 0, -6.18876, 0.315321, -1.44293, -7.40656, 0.911733, -0.714391, -3.94443, -2.62116, 1.77291, -2.55152, -3.3062, 6.50714, 0.799999],[0.275267, 0.182883, -0.43553, 0.881401, -0.113849, 0.808517, -1.71781, 0.127028, 0.644221, -1.25341, 0.121734, 0.417804, -1.84527, 0.0451395, 0.367866, -1.20314, 0.212896, 0.946111, 0.0484797, -2.42343, 2.19092, 0.814198, 0, 0, 0, -5.26158, 4.58224, -8.15481, -3.00749, -0.253759, -3.71714, -0.592671, -2.57911, 1.56006, 2.46843, 4.82106, -3.14096, 0.829999],[0.275809, 0.168856, -0.475166, 0.863542, -0.123405, 0.821594, -1.70444, 0.121325, 0.629711, -1.30587, 0.131647, 0.390849, -1.83134, 0.0748481, 0.389865, -1.173, 0.160101, 0.950668, 0.0546872, -4.49262, 3.12456, 0.0721125, 0, 0, 0, 2.92278, -2.36884, 11.0929, 0.697908, -2.30249, -5.90766, 1.85562, -2.76321, 1.33855, 2.14754, -2.25936, 8.35182, 0.859999],[0.276262, 0.155701, -0.517949, 0.841122, -0.111145, 0.806298, -1.66597, 0.12744, 0.597813, -1.33727, 0.146133, 0.365334, -1.81747, 0.0914841, 0.375614, -1.09096, 0.197411, 0.961839, 0.0273055, -3.38675, 2.24606, 0.63598, 0, 0, 0, -1.60254, 0.313753, -0.837248, -0.179442, -3.33342, -0.577897, 0.533312, -2.20045, 1.23944, 0.438395, 0.6832, 7.1559, 0.889999],[0.276403, 0.146771, -0.547271, 0.823986, -0.140809, 0.816115, -1.63983, 0.120852, 0.56808, -1.32002, 0.145566, 0.347436, -1.8065, 0.0868371, 0.411243, -1.03169, 0.237618, 0.946029, 0.00543943, -2.79473, 1.704, 1.00782, 0, 0, 0, -3.10828, 0.771663, 6.11581, -0.727647, -2.5292, 2.48318, -0.0214343, -1.61285, 1.23848, -0.699063, 5.11643, 5.26617, 0.919999],[0.276277, 0.138393, -0.577222, 0.804775, -0.148131, 0.808007, -1.61678, 0.119403, 0.550456, -1.31109, 0.14759, 0.333765, -1.79978, 0.0859837, 0.464787, -0.9669, 0.252857, 0.91635, -0.0244335, -3.38102, 1.91589, 0.802926, 0, 0, 0, 0.174948, -1.26368, 1.32418, 0.195046, -1.26733, -0.512731, 0.729853, -1.36438, 0.847845, 0.339802, 5.32104, 7.04067, 0.949999],[0.275972, 0.130315, -0.609489, 0.782011, -0.143736, 0.796457, -1.60269, 0.125354, 0.533277, -1.30283, 0.155426, 0.323565, -1.79685, 0.0943985, 0.52411, -0.915243, 0.269873, 0.904163, -0.0368338, -3.74536, 1.80513, 0.746808, 0, 0, 0, 0.611566, -1.08171, 1.52165, 0.915108, -2.33572, 2.6745, 0.799341, -0.808373, 0.0353619, 1.12008, 6.38956, 3.43415, 0.979999],[0.275524, 0.123216, -0.640879, 0.757689, -0.140045, 0.785791, -1.58278, 0.135496, 0.500517, -1.25167, 0.161992, 0.31567, -1.7943, 0.10666, 0.593237, -0.916282, 0.284987, 0.900776, -0.0566119, -3.60093, 1.71083, 0.644018, 0, 0, 0, 0.0683567, -0.890766, 2.42851, 1.11217, -3.62046, 5.92806, 0.554187, -0.749495, 0.269852, 1.15677, 6.14024, -0.0049776, 0.00999904],[0.274681, 0.114523, -0.671841, 0.731788, -0.163301, 0.800961, -1.65219, 0.138301, 0.434053, -1.12418, 0.168783, 0.306253, -1.79131, 0.135189, 0.712617, -1.07995, 0.253915, 0.873139, -0.0920219, -3.89578, 2.15137, 0.767666, 0, 0, 0, -4.01319, 2.73164, -13.5016, -0.126734, -8.46755, 16.465, 0.875435, -1.25232, 0.473287, 4.31971, 14.374, -23.3938, 0.039999],[0.273656, 0.10125, -0.709464, 0.69743, -0.18291, 0.806134, -1.75027, 0.157934, 0.397376, -1.09334, 0.187576, 0.290722, -1.78566, 0.178746, 0.813285, -1.22905, 0.240873, 0.844757, -0.113182, -5.24235, 2.77598, 0.164518, 0, 0, 0, 0.325024, -1.50805, -4.88693, 3.50917, 0.291607, -7.73698, 2.55272, -1.68771, 0.60593, 3.67312, 6.25111, -7.14648, 0.069999],[0.272524, 0.0867166, -0.75417, 0.650929, -0.164124, 0.793091, -1.76927, 0.198258, 0.395921, -1.1701, 0.222297, 0.27416, -1.78033, 0.225025, 0.868032, -1.28739, 0.24085, 0.83618, -0.112515, -6.4209, 2.9622, 0.2764, 0, 0, 0, 1.98292, 0.0411866, -1.69732, 3.69941, -2.24683, -2.85643, 3.92455, -1.55336, 0.394538, 6.02527, 6.70732, -8.44871, 0.099999],[0.271383, 0.0726667, -0.795038, 0.602191, -0.165866, 0.818787, -1.80008, 0.219245, 0.363716, -1.1463, 0.253344, 0.261125, -1.77589, 0.276326, 0.922525, -1.33898, 0.267689, 0.862272, -0.120886, -5.22804, 2.61924, 0.680591, 0, 0, 0, -2.00181, 4.58731, -4.48004, 0.868289, -3.77093, 4.48359, 2.87878, -1.39273, 0.600623, 4.92287, 5.12801, -1.55602, 0.129999],[0.270158, 0.0585876, -0.827486, 0.558422, -0.202977, 0.889169, -1.82486, 0.224619, 0.345368, -1.13128, 0.279777, 0.24752, -1.76869, 0.31784, 0.960103, -1.30551, 0.296751, 0.859216, -0.128078, -4.98843, 2.48495, 0.931674, 0, 0, 0, -3.46083, 7.13578, 0.457012, 0.580333, 0.282482, -1.26917, 2.33814, -1.10406, 0.613022, 3.23716, 2.57135, 5.78966, 0.159999],[0.268872, 0.0453058, -0.858473, 0.510854, -0.233681, 0.965051, -1.79776, 0.228181, 0.334243, -1.10632, 0.303989, 0.238622, -1.76453, 0.355172, 0.998424, -1.28598, 0.293573, 0.852175, -0.133484, -5.56113, 2.41504, 1.21756, 0, 0, 0, -2.76927, 9.24758, 4.27952, 0.240583, -2.75094, 6.19709, 2.9301, -0.777406, 0.359664, 4.91585, 5.64013, -1.43114, 0.189999],[0.267521, 0.0310341, -0.888802, 0.457239, -0.260872, 1.07183, -1.76689, 0.230286, 0.303109, -1.04592, 0.337154, 0.230593, -1.75996, 0.410686, 1.05799, -1.31056, 0.284484, 0.836578, -0.139843, -5.91319, 2.68367, 1.07925, 0, 0, 0, -3.34131, 11.5883, 1.27092, 0.229785, -2.63763, 4.01855, 3.5232, -1.03221, 0.589979, 5.70477, 5.39058, -2.55577, 0.219999]])

gzbLog = np.loadtxt(f'{BASE_CHECKPOINT_PATH}/MClogs/log_data_2.csv', delimiter=',')
acSet = gzbLog[:,39:51]
obSet = gzbLog[:,1:39]

print(obSet.shape)
for i in range(obSet.shape[1]):
    ob = obSet[i,:]
    ac = acSet[i,:]
    ob = ((ob-obMean)/obStd).astype(np.float32)
    action_ll = actor.architecture(torch.from_numpy(ob).cpu())
    print("pyt:", action_ll.detach().numpy()*actionStd+actionMean)
    print("Gzb:", ac)
    print("Dif:", action_ll.detach().numpy()*actionStd+actionMean-ac)

(635, 38)
pyt: [-0.17692828  0.53809703 -1.78437419 -0.06982759  0.78802061 -1.44685457
  0.0357151   0.81967196 -1.31020936  0.10708066  0.83255187 -1.77000563]
Gzb: [-0.176967   0.538115  -1.78437   -0.0698286  0.788115  -1.44686
  0.0356855  0.819752  -1.31018    0.107147   0.832582  -1.77001  ]
Dif: [ 3.87152357e-05 -1.79700756e-05 -4.18937683e-06  1.01358757e-06
 -9.43921709e-05  5.42651176e-06  2.96001692e-05 -8.00428052e-05
 -2.93636990e-05 -6.63370044e-05 -3.01332302e-05  4.36855316e-06]
pyt: [-0.15744669  0.55418796 -1.77036147 -0.05557681  0.7740877  -1.4678567
  0.02904765  0.79357305 -1.34164121  0.07761467  0.83138278 -1.74842703]
Gzb: [-0.15748    0.554212  -1.77035   -0.0555904  0.77418   -1.46786
  0.0290183  0.793639  -1.34162    0.0776718  0.831397  -1.74846  ]
Dif: [ 3.33145666e-05 -2.40405674e-05 -1.14711761e-05  1.35867790e-05
 -9.22967720e-05  3.30077171e-06  2.93471186e-05 -6.59468188e-05
 -2.12055492e-05 -5.71319883e-05 -1.42187328e-05  3.29725361e-05]
pyt: [-0.

(73, 38)


NameError: name 'plt' is not defined